# File Description
The file "2015-2019 Nasdaq.txt" contains the date, open, high, low, closing, adjusted close and volume values of the Nasdaq composite index from 02/01/2015 to 30/12/2019.

# Main Goal

1. Calculate the time series of log-returns associated to the daily closing values of the Nasdaq composite index.

2. Fit the best GARCH-type model to the time series of log-returns. Discuss your model fitting in a step-by-step fashion.


# 1. Libraries

In [ ]:
library(readr)
library(forecast)
library(fpp)
library(tseries)
library(fGarch)
library(repr)
library(car)

In [ ]:
options(repr.plot.width = 6,repr.plot.height = 4)

# 2. Import dataset

In [ ]:
df <- read_csv("2015-2019 Nasdaq.txt")

# 3. Exploratory Data Analysis

## 3.1 Initial Analysis

In [ ]:
summary(df)

In [ ]:
#Class Check
class(df)

In [ ]:
#Head and Tail
head(df)
tail(df)

In [ ]:
#Number of rows
nrow(df)

In [ ]:
#Checking for NAs
any(is.na(df))

In [ ]:
#Number of NAs
sum(is.na(df))

In [ ]:
df

## 3.2 Creating the log-return

- The log return was created using the close values.

In [ ]:
df.log_returns <- diff(log(df$Close), lag=1)

In [ ]:
ts.log_returns <- ts(df.log_returns, start=c(2015, 55), frequency=252) #number of trading days per year = 252

In [ ]:
plot.ts(ts.log_returns, xlab="Date", ylab="Log Returns")

## 3.3 Differencing

- It will be tested if there is a need for ARIMA differencing and SEASONAL differencing by using forecast library.

- ARIMA differencing function: https://www.rdocumentation.org/packages/forecast/versions/8.12/topics/ndiffs 

- SEASONAL differencing function: https://www.rdocumentation.org/packages/forecast/versions/8.12/topics/nsdiffs 

In [ ]:
ndiffs(ts.log_returns) 
nsdiffs(ts.log_returns)

# 3.4 Log-Returns  Analysis

### 3.4.1 Mean and Variance

In [ ]:
mean(ts.log_returns)

In [ ]:
var(ts.log_returns)

### 3.4.2 ACF and PACF

- Below are plotted the ACF and PACF of the log returns time series.

- Plus, ACF and PACF of the squared and absolute log returns.


In [ ]:
#ACF of Log_returns
Acf(ts.log_returns, lag.max=40, main="ACF Log Returns")
#ACF of Log_returns^2
Acf((ts.log_returns)^2, lag.max=40, main="ACF Squared Log Returns")
#ACF of absolute Log_returns
Acf(abs(ts.log_returns), lag.max=40, main="ACF Absolute Log Returns")
#PACF of Log_returns
Pacf(ts.log_returns, lag.max=40, main="PACF Log Returns", ylab="PACF")
#ACF of Log_returns^2
Pacf((ts.log_returns)^2, lag.max=40, main="PACF Squared Log Returns", ylab="PACF")
#ACF of absolute Log_returns
Pacf(abs(ts.log_returns), lag.max=40, main="PACF Absolute Log Returns", ylab="PACF")

### 3.4.3 Independence of the Log-Returns

- Box.test function: https://www.rdocumentation.org/packages/stats/versions/3.6.2/topics/Box.test

In [ ]:
Box.test(ts.log_returns,lag=7,type='Ljung')

#As we can not reject the null hypothesis (independence), we assume there is no serial dependence. 
#So we can now test if variance is constant or not.

In [ ]:
srr.log_returns=(ts.log_returns-mean(ts.log_returns))^2 #standardized squared residuals of log-returns
Box.test(srr.log_returns,lag=7,type='Ljung')

#It shows we can reject the null hypothesis (independence) on variance, so it has significant serial correlation.
#Therefore, there is an ARCH effect.

#### 3.4.4 ARIMA-type model fit
- auto_arima function: https://www.rdocumentation.org/packages/forecast/versions/8.12/topics/auto.arima

In [ ]:
model.arima <- auto.arima(ts.log_returns, seasonal=TRUE , allowmean=TRUE, approximation=TRUE,  stepwise=TRUE)

In [ ]:
print('-------------------------')
summary(model.arima)
print('-------------------------')
coeftest(model.arima)
print('-------------------------')

# 4. GARCH-type model fit

- garchFit function: https://www.rdocumentation.org/packages/fGarch/versions/3042.83.2/topics/garchFit

## 4.1 ARCH Model

In [ ]:
m_arch<-garchFit(~garch(1,0), ts.log_returns, trace=FALSE, cond.dist="std", include.delta=FALSE)

In [ ]:
m_arch<-garchFit(~garch(2,0), ts.log_returns, trace=FALSE, cond.dist="std", include.delta=FALSE)

In [ ]:
m_arch<-garchFit(~garch(3,0), ts.log_returns, trace=FALSE, cond.dist="std", include.delta=FALSE)

In [ ]:
m_arch<-garchFit(~garch(4,0), ts.log_returns, trace=FALSE, cond.dist="std", include.delta=FALSE)

In [ ]:
m_arch<-garchFit(~garch(5,0), ts.log_returns, trace=FALSE, cond.dist="std", include.delta=FALSE)

### 4.1.1 Residual Summary of the ARCH model

In [ ]:
summary(m_arch)

## 4.2 GARCH Model

In [ ]:
m_garch<-garchFit(~garch(1,1), ts.log_returns, trace=FALSE, cond.dist="std", include.delta=FALSE)

In [ ]:
m_garch<-garchFit(~garch(1,2), ts.log_returns, trace=FALSE, cond.dist="std", include.delta=FALSE)

In [ ]:
m_garch<-garchFit(~garch(1,3), ts.log_returns, trace=FALSE, cond.dist="std", include.delta=FALSE)

In [ ]:
m_garch<-garchFit(~garch(1,4), ts.log_returns, trace=FALSE, cond.dist="std", include.delta=FALSE)

In [ ]:
m_garch<-garchFit(~garch(2,1), ts.log_returns, trace=FALSE, cond.dist="std", include.delta=FALSE)

In [ ]:
m_garch<-garchFit(~garch(2,2), ts.log_returns, trace=FALSE, cond.dist="std", include.delta=FALSE)

In [ ]:
m_garch<-garchFit(~garch(2,3), ts.log_returns, trace=FALSE, cond.dist="std", include.delta=FALSE)

In [ ]:
m_garch<-garchFit(~garch(2,4), ts.log_returns, trace=FALSE, cond.dist="std", include.delta=FALSE)

In [ ]:
m_garch<-garchFit(~garch(3,1), ts.log_returns, trace=FALSE, cond.dist="std", include.delta=FALSE)

In [ ]:
m_garch<-garchFit(~garch(3,2), ts.log_returns, trace=FALSE, cond.dist="std", include.delta=FALSE)

In [ ]:
m_garch<-garchFit(~garch(3,3), ts.log_returns, trace=FALSE, cond.dist="std", include.delta=FALSE)

In [ ]:
m_garch<-garchFit(~garch(3,4), ts.log_returns, trace=FALSE, cond.dist="std", include.delta=FALSE)

In [ ]:
m_garch<-garchFit(~garch(4,1), ts.log_returns, trace=FALSE, cond.dist="std", include.delta=FALSE)

In [ ]:
m_garch<-garchFit(~garch(4,2), ts.log_returns, trace=FALSE, cond.dist="std", include.delta=FALSE)

In [ ]:
m_garch<-garchFit(~garch(4,3), ts.log_returns, trace=FALSE, cond.dist="std", include.delta=FALSE)

In [ ]:
m_garch<-garchFit(~garch(4,4), ts.log_returns, trace=FALSE, cond.dist="std", include.delta=FALSE)

### 4.2.1 Residual Summary of the GARCH model

In [ ]:
summary(m_garch)

## 4.3 APARCH Model

In [ ]:
m_aparch<-garchFit(~aparch(1,0), ts.log_returns, trace=FALSE, cond.dist="std", include.delta=TRUE)

In [ ]:
m_aparch<-garchFit(~aparch(2,0), ts.log_returns, trace=FALSE, cond.dist="std", include.delta=TRUE)

In [ ]:
m_aparch<-garchFit(~aparch(3,0), ts.log_returns, trace=FALSE, cond.dist="std", include.delta=TRUE)

In [ ]:
m_aparch<-garchFit(~aparch(4,0), ts.log_returns, trace=FALSE, cond.dist="std", include.delta=TRUE)

In [ ]:
m_aparch<-garchFit(~aparch(1,1), ts.log_returns, trace=FALSE, cond.dist="std", include.delta=TRUE)

In [ ]:
m_aparch<-garchFit(~aparch(1,2), ts.log_returns, trace=FALSE, cond.dist="std", include.delta=TRUE)

In [ ]:
m_aparch<-garchFit(~aparch(1,3), ts.log_returns, trace=FALSE, cond.dist="std", include.delta=TRUE)

In [ ]:
m_aparch<-garchFit(~aparch(1,4), ts.log_returns, trace=FALSE, cond.dist="std", include.delta=TRUE)

In [ ]:
m_aparch<-garchFit(~aparch(2,1), ts.log_returns, trace=FALSE, cond.dist="std", include.delta=TRUE)

In [ ]:
m_aparch<-garchFit(~aparch(2,2), ts.log_returns, trace=FALSE, cond.dist="std", include.delta=TRUE)

In [ ]:
m_aparch<-garchFit(~aparch(2,3), ts.log_returns, trace=FALSE, cond.dist="std", include.delta=TRUE)

In [ ]:
m_aparch<-garchFit(~aparch(2,4), ts.log_returns, trace=FALSE, cond.dist="std", include.delta=TRUE)

### 4.3.1 Residual Summary of the APARCH model

In [ ]:
summary(m_aparch)

## 4.4 Diagnostic of the chosen model residuals

In [ ]:
fit<-garchFit(~aparch(1,1), ts.log_returns, trace=FALSE, cond.dist="std", include.delta=TRUE)

In [ ]:
resid=residuals(fit)

In [ ]:
summary(resid)

- Plot Function: https://cran.r-project.org/web/packages/fGarch/fGarch.pdf

In [ ]:
plot(fit, which=11, main=".")

In [ ]:
qqPlot(resid)